In [1]:
%load_ext autoreload
%autoreload 2

A simple demonstration of a Route on folium map.

### Make sure you are viewing this on [nbviewer](https://nbviewer.jupyter.org/github/dumbPy/project_lakshadweep_itinerary/blob/master/Notebooks/how_it_works.ipynb)

In [2]:
import os
os.chdir('..')
from lakport_itinerary import inlandPorts, params, parse_schedule, generateGraph
from utils import RouteFinder
from datetime import datetime
import numpy as np
import folium

# Step 1 - Clean the Schedule

The Original Schedule Looks like


![](../Images/original_schedule.png)

Here, We see that each row represents a date, and there are multiple entries for each dates. Each column represents the schedule for a certain ship. Hence, if a ship reaches multiple Islands in a single day, they are in the same row. These type of schedule cannot be directly used to deduce any itineraries and it's required that the schedule be converted into a format that can be parsed by an algorithm. Hence, we convert this schedule into a `pandas.DataFrame` with each row containing a single entry, as below

In [3]:
parse_schedule('Lakport_schedule.html').head(20)

,Date,Amindivi,Arabian Sea,Bharat Seema,Corals,Kavaratti,Lagoons,Lakshadweep Sea,Minicoy
0,2019-03-15 13:00:00,0,Kochi,0,0,0,0,0,0
1,2019-03-16 09:00:00,0,Kiltan,0,0,0,0,0,0
2,2019-03-16 13:00:00,0,0,0,0,0,0,0,Beypore
3,2019-03-16 14:00:00,0,Chetlat,0,0,0,0,0,0
4,2019-03-16 18:00:00,0,Bitra,0,0,0,0,0,0
5,2019-03-17 09:00:00,0,Kadmat,0,0,0,0,0,0
6,2019-03-17 10:00:00,0,0,0,0,0,0,0,Amini
7,2019-03-17 12:30:00,0,Amini,0,0,0,0,0,0
8,2019-03-17 12:45:00,0,0,0,0,0,0,Kochi,0
9,2019-03-17 14:00:00,0,0,0,0,0,0,0,Kadmat


Here, each row represents the position of a perticular ship at a certain date and time.
Each row hence represents a Node of a network, representing presence of a perticular ship at perticular date and time at a perticular location.
This `pandas.DataFrame` can now be iterated upon, one row (or Node) at a time by an algorithm to find all the Nodes that can be reached by a source Node, subject to some constraints 

A good idea of what a itinerary can be like is as below.

Here, Yellow edge represents staying at a location, hence the location at the edge's start and end remain same. On the other hand, the date and time changes and the ship might change.

Red edge represent travelling between two locations by a perticular ship, henc the ship remains the same and the date and time change.

We start from a Mainland India port and return to a mainland India port.

![](../Images/Manual Itinerary Sample.jpg)

There are three Mainland India ports, __Kochi__, __Mangalore__ and __Beypore__ and the rest are Islands

# Step 2 - Generating Graph

Graph is generated subject to some conditions stated in `params`  

In [4]:
params = {
'maxDaysOnOneIsland' : 5, # Maximum days of stay on one Island
'duration' : 15, # Max duration of tour
'Departure' : datetime.strptime('11/03/2018', '%d/%m/%Y'), # Earliest start date
'source' : inlandPorts, # Tour starts from. any of the inland Ports
'destination' : inlandPorts, # Tour ends at any of the ports, or provide list of ports
'minHoursOnOneIsland' : 3,
'maxHoursPerShip' : 24,      
'max_n_routes' : np.infty,   
'html_file' : 'Lakport_schedule.html', #parse this file for schedule above
'filler' : 0 # Used to fill NaN values
}

Nodes are defined in `utils.py` module as [locationNode](https://github.com/dumbPy/project_lakshadweep_itinerary/blob/62ae259461b21466fd944dbb71008151eb9c303c/utils.py#L13), and have two important attributes, `location` and `timestamp`.

As each row represents a row, we can represent the schedule as a set of nodes numbered $1$ to $n$ where the cleaned schedule has $n$ rows.

As the rows are sorted by date and time, (`datetime.datetime` in python),  
set of nodes $N = \{N_i : i\in [1,2,..n]\}$

Now that we have the nodes, we evaluate the possibility of edges between them, and edges that satisfy the below conditions are added to the graph. [`check_condition(node1, node2)`](https://github.com/dumbPy/project_lakshadweep_itinerary/blob/62ae259461b21466fd944dbb71008151eb9c303c/lakport_itinerary.py#L74) does just that

### Condition 0: (Compulsary)
`node1.timestamp < node2.timestamp` as we can only go forward in time

### Conditions Set 1 (Edge representing Stay at some Island)

`node1.location == node2.location` Location of both nodes should be same __and__  
duration of stay should be within `minHoursOnOneIsland` and `maxDaysOnOneIsland`

##### Stay edge satisfying above condition should be on island only
stay edge satisfying above condition 1 should also satisfy `node1.location in inlandPorts`

### Condition Set 2 (Edge representing Travel)
`node1.ship == node2.ship` i.e., ship leaving node1 should be reaching node2 __and__  
duration of travel should be less than `maxHoursPerShip`

# Step 3 - Finding Routes

Transversing the Graph and finding all routes starting on `source` nodes and ending at `destination` nodes

In an Itinerary, there are 2 types of edges, _Travel Edges_ representing travel between two nodes and _Stay Edges_ representing stay between two nodes.  
`RouteFinder` class is responsible for finding all possible valid routes.  
It uses two helper functions, `EdgeType(node1, node2)` to determine the type of edge between the 2 nodes and `addItinerary(nodes)` to add itinerary to its list of routes from the given set of nodes.

#### `walk(priv_path)`

This method takes previous path transversed by the algorithm, and finds the next nodes that are reachable from current node, subject to edge type, i.e., if last edge was a _Travel Edge_, find all possible _Stay Edges_ originating from current node, or if last edge was _Stay Edge_, find all possible _Travel Edges_ originating from current node.  


This is a recursive algorithm that either reaches a possible destination in which case the transversed path from source to destination is added as a possible route, or ends up at a dead end.

# Showing Routes

In [5]:
params

{'maxDaysOnOneIsland': 5,
 'duration': 15,
 'Departure': datetime.datetime(2018, 3, 11, 0, 0),
 'source': ['Kochi', 'Mangalore', 'Beypore'],
 'destination': ['Kochi', 'Mangalore', 'Beypore'],
 'minHoursOnOneIsland': 3,
 'maxHoursPerShip': 24,
 'max_n_routes': inf,
 'html_file': 'Lakport_schedule.html',
 'filler': 0}

In [6]:
inlandPorts

['Kochi', 'Mangalore', 'Beypore']

In [7]:
schedule = parse_schedule(**params)
G = generateGraph(schedule, **params)
routes = RouteFinder.find_routes(G, **params)

Total Nodes Added:  88
Total Routes Found: 22


In [8]:
routes[0].draw()

In [10]:
# You can also print the itinerary like below
routes[18].draw(print_=True)

Kochi 2019-03-21 12:00:00
{'ship': 'Lakshadweep Sea 0 days 22:00:00'} 0 days 22:00:00
Agatti 2019-03-22 10:00:00
{'ship': 'None 3 days 00:00:00'} 3 days 00:00:00
Agatti 2019-03-25 10:00:00
{'ship': 'Corals 0 days 08:00:00'} 0 days 08:00:00
Kavaratti 2019-03-25 18:00:00
{'ship': 'None 4 days 00:00:00'} 4 days 00:00:00
Kavaratti 2019-03-29 18:00:00
{'ship': 'Arabian Sea 0 days 14:00:00'} 0 days 14:00:00
Kochi 2019-03-30 08:00:00
Tour Duration:  8 days 20:00:00



### The routes can also bee subjected to some conditions.
### Say I want to travel atleast for 10 days

In [11]:
from datetime import timedelta
routes2 = [route for route in routes if route.nodes[-1].timestamp-route.nodes[0].timestamp>timedelta(days=10)]
len(routes2)

1

In [12]:
routes2[0].draw(print_=True)

Kochi 2019-03-18 13:00:00
{'ship': 'Corals 0 days 20:00:00'} 0 days 20:00:00
Kavaratti 2019-03-19 09:00:00
{'ship': 'None 1 days 00:00:00'} 1 days 00:00:00
Kavaratti 2019-03-20 09:00:00
{'ship': 'Kavaratti 0 days 05:00:00'} 0 days 05:00:00
Agatti 2019-03-20 14:00:00
{'ship': 'None 4 days 20:00:00'} 4 days 20:00:00
Agatti 2019-03-25 10:00:00
{'ship': 'Corals 0 days 08:00:00'} 0 days 08:00:00
Kavaratti 2019-03-25 18:00:00
{'ship': 'None 4 days 00:00:00'} 4 days 00:00:00
Kavaratti 2019-03-29 18:00:00
{'ship': 'Arabian Sea 0 days 14:00:00'} 0 days 14:00:00
Kochi 2019-03-30 08:00:00
Tour Duration:  11 days 19:00:00

